# Hydrometrics2

In this notebook, we show how to compute several **hydrometics parameters** based on stream network produced from  model. The analysis relies on the `flow` files (*i.e.* stream) found in Badlands outputs. If you are interested in looking at morphometrics and stratigraphic analysis there are other notebooks specially designed for that in the **Badlands companion** repository.

**Hydrometrics** here refers only to quantitative description and analysis of water surface and we don't consider groundwater analysis. We will show how you can extract a particular catchment from a given model and compute for this particular catchment a series of paramters such as:

- **river profile** evolution based on main stream elevation and distance to outlet,
- **peclet number** distribution which evaluates the dominant processes shaping the landscape,
- $\chi$ **parameter** that characterizes rivers system evolution based on terrain steepness and the arrangement of tributaries,
- **discharge** profiles 

In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import cm

# Import badlands grid generation toolbox
import hydroGrid as hydr

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

# 1. Load catchments parameters

We first have to define the path to the Badlands outputs we want to analyse. In addition Badlands is creating several files for each processors that have been used, you need to specify this number in the `ncpus` variable. 

We then need to provide a point coordinates (X,Y) contained in the catchment of interest. This point doesn't need to be the outlet of the catchment. 

For more information regarding the function uncomment the following line.

In [ ]:
#help(hydr.hydroGrid.__init__)

In [ ]:
hydro = hydr.hydroGrid(folder='/workspace/volume/transtension/outputs/output_0.50_0.50_kd1e-5/h5/', ncpus=1, \
                          ptXY = [53000,30000])

# 2. Extract particular catchment dataset

We now extract the data from a particular time step (`timestep`) and for the catchment of interest, which contained the point specified in previous function.

**Note**

If you are interested in making some hydrometric comparisons between different time steps you can create multiple instances of the hydrometrics python class each of them associated to a given time step.

In [ ]:
#help(hydro.getCatchment)

In [ ]:
hydro.getCatchment(timestep=40)

We can visualise the stream network using the `viewNetwork` function. The following paramters can be displayed:
- $\chi$ paramater 'chi',
- elevation 'Z',
- discharge 'FA' (logarithmic values)

In [ ]:
#help(hydro.viewNetwork)

In [ ]:
#hydro.viewNetwork(markerPlot = True, linePlot = False, lineWidth = 1, markerSize = 3, 
#                 val = 'FA', width = 1000, height = 600, colorMap = cm.Blues, 
#                 colorScale = 'Blues', reverse = True, 
#                 title = '<br>Stream network graph')

# 3. Extract main stream for the considered catchment

We now extract the main stream for the considered catchment based on flow 
discharge values.

In [ ]:
#help(hydro.extractMainstream)

In [ ]:
hydro.extractMainstream()

As for the global stream network, you can use the `viewStream` function to visualise the main stream dataset.

In [ ]:
#help(hydro.viewStream)

In [ ]:
#hydro.viewStream(linePlot = False, lineWidth = 1, markerSize = 7, 
#                 val = 'Z', width = 1000, height = 600, colorMap = cm.jet, 
#                 colorScale = 'Jet', reverse = False, 
#                 title = '<br>Stream network graph')

# 4. Compute main stream hydrometrics

Here, we compute the stream parameters using the distance from outlet and the Badlands simulation coefficients for the **stream power law** and the **hillslope linear diffusion**.

The formulation for the Peclet number is: 

$$Pe =\frac {\kappa_{c}l^{2(m+1)-n}}{\kappa_{d}z^{1-n}}$$

where $\kappa_{c}$ is the erodibility coefficient, $\kappa_{d}$ the hillslope diffusion coefficient and *m*, *n* the exponents from the stream power law equation. Their values are defined in your model input file.

The formulation for the $\chi$ parameter follows:

$$\chi = \int_{x_b}^x \left( \frac{A_o}{A(x')} \right)^{m/n} dx' $$

where $A_o$ is an arbitrary scaling area, and the integration is performed upstream from base level to location $x$.
         
In addition the function `computeParams` requires an additional parameter `num` which is the number of samples to generate along the main stream profile for linear interpolation.

In [ ]:
#help(hydro.computeParams)

In [ ]:
hydro.computeParams(kd=0.01, kc=5.e-6, m=0.5, n=1., num=500)

The following combination of parameters can be visualised with the `viewPlot` function:
- 'dist': distance from catchment outlet
- 'FA': flow discharge (logorithmic)
- 'Pe': Peclet number
- 'chi': $\chi$ parameter
- 'Z': elevation from outlet.

In [ ]:
#help(hydro.viewPlot)

In [ ]:
hydro.viewPlot(lineWidth = 3, markerSize = 5, xval = 'dist', yval = 'chi',
               width = 600, height = 500, colorLine = 'black', colorMarker = 'black',
               opacity = 0.2, title = 'Chi vs distance to outlet')

# 5. Extract all stream positions for the catchment

Here we define 2 numpy arrays __donorPts__ and __rcvPts__ which are the X,Y coordinates of the catchment points. 

They are __ordered__ so each tuple of points is connected to form the drainage. 
Each of these segments represents a small stream. 

In [ ]:
donorPts = np.zeros((len(hydro.donX),2))
donorPts[:,0] = hydro.donX
donorPts[:,1] = hydro.donY

rcvPts = np.zeros((len(hydro.donX),2))
rcvPts[:,0] = hydro.rcvX
rcvPts[:,1] = hydro.rcvY

The $k^{th}$ stream coordinates are given by the segment (AB)$_k$ with A and B the upstream and downstream points respectively by:

```python
Ax = donorPts[k,0]
Ay = donorPts[k,1]
Bx = rcvPts[k,0]
By = rcvPts[k,1]
```

# 6. Define tributary streams

First we find unique elements in the donors and receivers list. These elements are the top (leaves) of the tributaries (in the network graph). 

__This function might take a while if you have a lot of small tributaries in your catchment !__

In [ ]:
#help(hydro.findTribTop)
topIDs = hydro.findTribTop(donor = donorPts, rcv = rcvPts)

#help(hydro.extractAllStreams)
streamList = hydro.extractAllStreams(startIDs = topIDs)

# 7. Compute all tributaries hydrometrics

Here, we compute the stream parameters using the distance from outlet and the Badlands simulation coefficients for the **stream power law** and the **hillslope linear diffusion**.

The formulation for the Peclet number is: 

$$Pe =\frac {\kappa_{c}l^{2(m+1)-n}}{\kappa_{d}z^{1-n}}$$

where $\kappa_{c}$ is the erodibility coefficient, $\kappa_{d}$ the hillslope diffusion coefficient and *m*, *n* the exponents from the stream power law equation. Their values are defined in your model input file.

The formulation for the $\chi$ parameter follows:

$$\chi = \int_{x_b}^x \left( \frac{A_o}{A(x')} \right)^{m/n} dx' $$

where $A_o$ is an arbitrary scaling area, and the integration is performed upstream from base level to location $x$.
         
In addition the function `computeParams` requires an additional parameter `num` which is the number of samples to generate along the main stream profile for linear interpolation.

In [ ]:
#help(hydro.computeTribParams)
hydro.computeTribParams(tribList=streamList, kd=0.01, kc=5.e-6, m=0.5, n=1., num=100)

The following combination of parameters can be visualised with the `viewPlot` function:
- 'dist': distance from catchment outlet
- 'FA': flow discharge (logorithmic)
- 'Pe': Peclet number
- 'chi': $\chi$ parameter
- 'Z': elevation from outlet.

For defining your own colors you can use RGBA values (have a look [here](http://www.hexcolortool.com/) to easely pick a color).

In [ ]:
#help(hydro.viewTribPlot)
hydro.viewTribPlot(tribWidth = 2, tribColor = 'rgba(207, 215, 222, 0.5)', mainWidth = 5, 
                   mainColor = 'rgba(233, 109, 196, 1)', xval = 'dist', yval = 'chi', 
                   tribList = streamList, distLimit = 50000,
                   width = 800, height = 600, title = 'distance vs elevation')